In [259]:
# Step 1: Filter and Clean Invoice Data
import pandas as pd


# 🔧 Configure which sites to process
selected_sites = ["DIT", "SPN", "SPCP"]  # Example: update these as needed

# Load the invoice input data
invoice_path = "invoice_input_data_all.xlsx"  # Update path if needed
invoice_df = pd.read_excel(invoice_path)
print(invoice_df.shape)
invoice_df = invoice_df[invoice_df['model'] == True]
print(invoice_df.shape)

invoice_df = invoice_df[[
    "invoice_id", "site",'invoice_commodity_quantity', "invoice_commodity_group", "invoice_commodity_description",
    "location", "model", "unit", "rate_unit", "freight_class", "applied_rate",
    "shipment_type", "realistic_optimal_method", "xgs_rate", "historical_rate"
]]

invoice_df["invoice_commodity_description"] = invoice_df["invoice_commodity_description"].apply(
    lambda x: x.title() if str(x).strip().lower() == "carpet tiles" else x
)

# Filter input invoices to selected sites
invoice_df = invoice_df[invoice_df["site"].isin(selected_sites)]


(17846, 61)
(13048, 61)


In [260]:
invoice_df['invoice_commodity_description'].unique()

array(['VCT', 'Carpet Tiles', 'LVP', 'LVT', 'Carpet Roll'], dtype=object)

In [261]:
# Step 2: Pivot Both Arithmetic and Weighted Averages

from numpy import average

freight_class_columns = ['L5C', '5C', '1M', '2M', '3M', '5M', '10M', '20M', '30M', '40M']

# First, filter again just to be safe
invoice_filtered = invoice_df[
    invoice_df["freight_class"].notna() &
    invoice_df["historical_rate"].notna() &
    invoice_df["xgs_rate"].notna() &
    invoice_df["invoice_commodity_quantity"].notna()
][[
    "site", 
    "rate_unit", 
    "invoice_commodity_group", 
    "invoice_commodity_description",
    "freight_class", 
    "historical_rate",
    "xgs_rate",
    "invoice_commodity_quantity"
]].copy()

# Step 2: Compute arithmetic and weighted averages

group_cols = ["site", "rate_unit", "invoice_commodity_group", "invoice_commodity_description", "freight_class"]

# Group invoice data
grouped = invoice_filtered.groupby(group_cols)

# Step 2A: Arithmetic averages using .agg()
summary_avg = grouped.agg(
    hist_avg=("historical_rate", "mean"),
    xgs_avg=("xgs_rate", "mean")
)

# Step 2B: Weighted averages using .apply()
def compute_wavg(grp):
    return pd.Series({
        "hist_wavg": average(grp["historical_rate"], weights=grp["invoice_commodity_quantity"]),
        "xgs_wavg": average(grp["xgs_rate"], weights=grp["invoice_commodity_quantity"])
    })

summary_wavg = grouped.apply(compute_wavg)

# Step 2C: Combine both summaries
summary = pd.concat([summary_avg, summary_wavg], axis=1).reset_index()

# Preview
print("✅ Summary with arithmetic and weighted averages:")
summary.head()

✅ Summary with arithmetic and weighted averages:


,site,rate_unit,invoice_commodity_group,invoice_commodity_description,freight_class,hist_avg,xgs_avg,hist_wavg,xgs_wavg
0,DIT,CWT,1VNL,LVP,1M,0.198404,0.174759,0.200220,0.174759
1,DIT,CWT,1VNL,LVP,2M,0.046007,0.138766,0.044963,0.138766
2,DIT,CWT,1VNL,LVP,3M,0.034314,0.138766,0.034314,0.138766
3,DIT,CWT,1VNL,LVP,5C,0.215293,0.232444,0.220168,0.232444
4,DIT,CWT,1VNL,LVP,L5C,0.875810,1.363826,0.499970,0.693883


In [262]:
summary['invoice_commodity_description'].unique()

array(['LVP', 'LVT', 'VCT', 'Carpet Roll', 'Carpet Tiles'], dtype=object)

In [263]:
# Step 4: Create block tables for each metric (no column prefixes)

index_cols = ["site", "rate_unit", "invoice_commodity_group", "invoice_commodity_description"]
freight_classes = ['L5C', '5C', '1M', '2M', '3M', '5M', '10M', '20M', '30M', '40M']

def safe_pivot(metric_col, source_name):
    pivoted = summary.pivot(index=index_cols, columns="freight_class", values=metric_col).reset_index()
    
    # Ensure all freight class columns are present
    for fc in freight_classes:
        if fc not in pivoted.columns:
            pivoted[fc] = None

    # Add required template columns
    pivoted.rename(columns={
        "rate_unit": "unit",
        "invoice_commodity_group": "commodity_group",
        "invoice_commodity_description": "commodity_description"
    }, inplace=True)
    pivoted["site_description"] = "Itasca"
    pivoted["unitclass"] = pivoted["unit"].apply(lambda x: "Weight" if x == "CWT" else "Area")
    pivoted["source"] = source_name

    # Reorder
    ordered_cols = ["site_description", "site", "unit", "unitclass", "commodity_group", "commodity_description"] + freight_classes + ["source"]
    return pivoted[ordered_cols]

# Generate four blocks
hist_avg_block = safe_pivot("hist_avg", "hist_avg")
xgs_avg_block = safe_pivot("xgs_avg", "xgs_avg")
hist_wavg_block = safe_pivot("hist_wavg", "hist_wavg")
xgs_wavg_block = safe_pivot("xgs_wavg", "xgs_wavg")

# Optionally combine all
combined_output = pd.concat([hist_avg_block, xgs_avg_block, hist_wavg_block, xgs_wavg_block], ignore_index=True)

# Sort for visual clarity
combined_output = combined_output.sort_values(by=["commodity_group", "commodity_description", "site", "unit", "source"]).reset_index(drop=True)

# Preview
print("✅ Combined pivot output (clean format):")
combined_output.head()


✅ Combined pivot output (clean format):


freight_class,site_description,site,unit,unitclass,commodity_group,commodity_description,L5C,5C,1M,2M,3M,5M,10M,20M,30M,40M,source
0,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,1.734967,1.021696,0.830599,2.816536,0.482738,0.165966,0.159801,NaN,NaN,NaN,hist_avg
1,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,1.148743,1.011448,0.826022,2.816536,0.482738,0.165966,0.159801,NaN,NaN,NaN,hist_wavg
2,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,0.986601,0.478556,0.469892,0.461337,0.452672,0.190923,0.200150,NaN,NaN,NaN,xgs_avg
3,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,0.571614,0.478555,0.469892,0.461337,0.452672,0.190923,0.200150,NaN,NaN,NaN,xgs_wavg
4,Itasca,SPCP,SQYD,Area,1CBL,Carpet Roll,5.904222,1.228246,1.078833,1.270228,1.048338,NaN,0.170476,NaN,NaN,NaN,hist_avg


In [264]:
# Step 3: Ensure All Required Columns in Combined Output

freight_classes = ['L5C', '5C', '1M', '2M', '3M', '5M', '10M', '20M', '30M', '40M']

# Ensure all freight class columns exist in the output
for col in freight_classes:
    if col not in combined_output.columns:
        combined_output[col] = None

# Ensure proper column order
ordered_cols = [
    "site_description", "site", "unit", "unitclass", "commodity_group", "commodity_description"
] + freight_classes + ["source"]

combined_output = combined_output[ordered_cols]

# Preview the cleaned, structured result
print("✅ Final Structured Invoice Summary (Step 3):")
combined_output.head()


✅ Final Structured Invoice Summary (Step 3):


freight_class,site_description,site,unit,unitclass,commodity_group,commodity_description,L5C,5C,1M,2M,3M,5M,10M,20M,30M,40M,source
0,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,1.734967,1.021696,0.830599,2.816536,0.482738,0.165966,0.159801,NaN,NaN,NaN,hist_avg
1,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,1.148743,1.011448,0.826022,2.816536,0.482738,0.165966,0.159801,NaN,NaN,NaN,hist_wavg
2,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,0.986601,0.478556,0.469892,0.461337,0.452672,0.190923,0.200150,NaN,NaN,NaN,xgs_avg
3,Itasca,DIT,SQYD,Area,1CBL,Carpet Roll,0.571614,0.478555,0.469892,0.461337,0.452672,0.190923,0.200150,NaN,NaN,NaN,xgs_wavg
4,Itasca,SPCP,SQYD,Area,1CBL,Carpet Roll,5.904222,1.228246,1.078833,1.270228,1.048338,NaN,0.170476,NaN,NaN,NaN,hist_avg


In [265]:
# Step 4: Add Source Column and Append to Vendor Data

# Load vendor data
vendor_path = "freight_rates_operating_multi_reporting_all.csv"  # Update path if needed
vendor_df = pd.read_csv(vendor_path)

# Filter vendor freight rates to selected sites
vendor_df = vendor_df[vendor_df["site"].isin(selected_sites)]


# Add source tag
vendor_df["source"] = "vendor"

# Ensure all required freight class columns exist in vendor_df
freight_classes = ['L5C', '5C', '1M', '2M', '3M', '5M', '10M', '20M', '30M', '40M']
for col in freight_classes:
    if col not in vendor_df.columns:
        vendor_df[col] = None

# Ensure consistent column ordering
final_cols = [
    "site_description", "site", "unit", "unitclass", "commodity_group", "commodity_description"
] + freight_classes + ["source"]

vendor_df = vendor_df[final_cols]
combined_output = combined_output[final_cols]  # Already structured in prior step

# Append invoice summary blocks to vendor table
combined_df = pd.concat([vendor_df, combined_output], ignore_index=True)

# Preview the result
print("✅ Appended Final Table (Step 4):")
combined_df.tail()


✅ Appended Final Table (Step 4):


,site_description,site,unit,unitclass,commodity_group,commodity_description,L5C,5C,1M,2M,3M,5M,10M,20M,30M,40M,source
70,Itasca,SPCP,CWT,Weight,1VNL,VCT,0.541678,0.241744,0.192654,0.160963,0.160962,0.12577,0.104687,0.073545,NaN,NaN,xgs_wavg
71,Itasca,SPN,CWT,Weight,1VNL,VCT,0.549503,0.143911,0.092570,0.068993,0.082354,NaN,0.048974,0.042739,0.027862,0.010330,hist_avg
72,Itasca,SPN,CWT,Weight,1VNL,VCT,0.358993,0.138733,0.091176,0.070136,0.083377,NaN,0.049539,0.042739,0.027570,0.010330,hist_wavg
73,Itasca,SPN,CWT,Weight,1VNL,VCT,0.929186,0.136070,0.082880,0.062586,0.062586,NaN,0.039936,0.025647,0.019882,0.020931,xgs_avg
74,Itasca,SPN,CWT,Weight,1VNL,VCT,0.515031,0.131530,0.082879,0.062585,0.062586,NaN,0.039521,0.025647,0.019865,0.020931,xgs_wavg


In [266]:
# Step 6: Normalize Vendor Rates from $/CWT to $/LBS

# Identify rows where unit is CWT (used for 1VNL)
vendor_cwt_mask = (combined_df["source"] == "vendor") & (combined_df["unit"] == "CWT")

# List of freight class columns to scale
freight_class_cols = ['L5C', '5C', '1M', '2M', '3M', '5M', '10M', '20M', '30M', '40M']

# Convert vendor rates from $/CWT to $/LBS
combined_df.loc[vendor_cwt_mask, freight_class_cols] = combined_df.loc[vendor_cwt_mask, freight_class_cols] / 100

print("✅ Converted vendor CWT rates to $/LBS for comparability.")


✅ Converted vendor CWT rates to $/LBS for comparability.


In [267]:
combined_df['source'].unique()

array(['vendor', 'hist_avg', 'hist_wavg', 'xgs_avg', 'xgs_wavg'],
      dtype=object)

In [268]:
# Step X: Append Variance Rows Between hist_invoice and xgs_invoice

# Columns used to match rows
index_cols = [
    "site_description", "site", "unit", "unitclass",
    "commodity_group", "commodity_description"
]

# Freight class columns to compute variance on
freight_class_cols = ['L5C', '5C', '1M', '2M', '3M', '5M', '10M', '20M', '30M', '40M']

# Separate historical and xgs rows
hist_df = combined_df[combined_df["source"] == "hist_wavg"]
xgs_df = combined_df[combined_df["source"] == "xgs_wavg"]

# Merge them on the index columns
variance_df = pd.merge(hist_df, xgs_df, on=index_cols, suffixes=("_hist", "_xgs"))

# Compute variance
variance_data = variance_df[index_cols].copy()
for col in freight_class_cols:
    variance_data[col] = variance_df[f"{col}_hist"] - variance_df[f"{col}_xgs"]

# Add source column
variance_data["source"] = "variance"

# Append to combined table
combined_df = pd.concat([combined_df, variance_data], ignore_index=True)

# Optional: sort for clarity
combined_df.sort_values(by=index_cols + ["source"], inplace=True)

# Preview result
print("✅ Variance rows added.")
combined_df.tail()


✅ Variance rows added.


,site_description,site,unit,unitclass,commodity_group,commodity_description,L5C,5C,1M,2M,3M,5M,10M,20M,30M,40M,source
11,Spectra Coppell,SPCP,CWT,Weight,1VNL,LVP,0.2653,0.2418,0.1927,0.1610,0.1610,0.1258,0.1057,0.1057,0.1057,0.1057,vendor
10,Spectra Coppell,SPCP,CWT,Weight,1VNL,LVT,0.2653,0.2418,0.1927,0.1610,0.1610,0.1258,0.1057,0.1057,0.1057,0.1057,vendor
12,Spectra Coppell,SPCP,CWT,Weight,1VNL,VCT,0.2653,0.2418,0.1927,0.1610,0.1610,0.1258,0.1057,0.1057,0.1057,0.1057,vendor
13,Spectra Coppell,SPCP,SQYD,Area,1CBL,Carpet Roll,0.5073,0.4892,0.4819,0.4747,0.4601,0.4601,0.4601,0.4601,0.4601,0.4601,vendor
14,Spectra Coppell,SPCP,SQYD,Area,1CPT,Carpet Tiles,0.8623,0.8316,0.8192,0.8070,0.7822,0.7822,0.7822,0.7822,0.7822,0.7822,vendor


In [269]:
combined_df['commodity_description'].unique()

array(['LVP', 'LVT', 'VCT', 'Carpet Roll', 'Carpet Tiles'], dtype=object)

In [270]:
# Define freight classes in correct order
freight_classes = ['L5C', '5C', '1M', '2M', '3M', '5M', '10M', '20M', '30M', '40M']

# Group by site, commodity, and freight class → count unique invoice_ids
invoice_counts = invoice_df.groupby(
    ["site", "invoice_commodity_description", "freight_class"]
)["invoice_id"].nunique().reset_index(name="invoice_count")

# List to collect all site-level matrices
invoice_matrix_list = []

# Loop over sites
for site in invoice_counts["site"].unique():
    site_df = invoice_counts[invoice_counts["site"] == site]

    # Pivot per site
    matrix = site_df.pivot_table(
        index="invoice_commodity_description",
        columns="freight_class",
        values="invoice_count",
        fill_value=0
    )

    # Reindex to ensure all freight classes are present
    matrix = matrix.reindex(columns=freight_classes, fill_value=0)
    matrix = matrix.reset_index()

    # Add required columns
    matrix["site_description"] = "Itasca"  # Adjust dynamically if needed
    matrix["site"] = site
    matrix["unit"] = None
    matrix["unitclass"] = None
    matrix["commodity_group"] = None
    matrix["source"] = "invoice_counts"

    # Reorder to match combined_df structure
    final = matrix[[
        "site_description", "site", "unit", "unitclass",
        "commodity_group", "invoice_commodity_description"
    ] + freight_classes + ["source"]]

    final.rename(columns={"invoice_commodity_description": "commodity_description"}, inplace=True)
    invoice_matrix_list.append(final)

# Concatenate all site-level matrices
invoice_matrix_final = pd.concat(invoice_matrix_list, ignore_index=True)

# ✅ Now append to combined_df
combined_df = pd.concat([combined_df, invoice_matrix_final], ignore_index=True)

combined_df


,site_description,site,unit,unitclass,commodity_group,commodity_description,L5C,5C,1M,2M,3M,5M,10M,20M,30M,40M,source
0,Itasca,DIT,CWT,Weight,1VNL,LVP,0.875810,0.215293,0.198404,0.046007,0.034314,NaN,NaN,NaN,NaN,NaN,hist_avg
1,Itasca,DIT,CWT,Weight,1VNL,LVP,0.499970,0.220168,0.200220,0.044963,0.034314,NaN,NaN,NaN,NaN,NaN,hist_wavg
2,Itasca,DIT,CWT,Weight,1VNL,LVP,-0.193913,-0.012276,0.025461,-0.093803,-0.104452,NaN,NaN,NaN,NaN,NaN,variance
3,Itasca,DIT,CWT,Weight,1VNL,LVP,0.282700,0.232500,0.174800,0.138800,0.138800,0.0985,0.0717,0.0717,0.0717,0.0717,vendor
4,Itasca,DIT,CWT,Weight,1VNL,LVP,1.363826,0.232444,0.174759,0.138766,0.138766,NaN,NaN,NaN,NaN,NaN,xgs_avg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Itasca,SPN,None,None,None,Carpet Roll,95.000000,5.000000,5.000000,1.000000,0.000000,2.0000,0.0000,0.0000,0.0000,0.0000,invoice_counts
101,Itasca,SPN,None,None,None,Carpet Tiles,657.000000,70.000000,21.000000,12.000000,8.000000,2.0000,1.0000,0.0000,0.0000,0.0000,invoice_counts
102,Itasca,SPN,None,None,None,LVP,98.000000,19.000000,38.000000,22.000000,9.000000,7.0000,7.0000,4.0000,1.0000,0.0000,invoice_counts
103,Itasca,SPN,None,None,None,LVT,134.000000,39.000000,42.000000,22.000000,18.000000,21.0000,6.0000,3.0000,0.0000,4.0000,invoice_counts


In [271]:
# Final Sort: Enforce output row order for readability

# Define source display order
source_order = {
       "invoice_counts": 0,  # Always shown last
    "vendor": 1,
    "hist_avg": 2,
    "hist_wavg": 3,
    "xgs_avg": 4,
    "xgs_wavg": 5,
    "variance": 6 , # Always shown last
}

# Add sorting key column
combined_df["source_sort"] = combined_df["source"].map(source_order)

# Sort rows to follow commodity hierarchy and defined source order
combined_df = combined_df.sort_values(
    by=["commodity_group", "commodity_description", "site", "unit", "source_sort"]
).drop(columns="source_sort")

# Reset index for cleanliness
combined_df.reset_index(drop=True, inplace=True)

print("✅ Rows sorted for visual clarity.")


✅ Rows sorted for visual clarity.


In [272]:
# 🔄 Save combined_df with each site as a separate Excel sheet

import pandas as pd

# Set export path
output_path = "freight_rates_by_site.xlsx"  # Change path if needed

# Get unique sites
sites = combined_df["site"].dropna().unique()

# Export to Excel with one sheet per site
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    for site in sites:
        sheet_name = str(site)[:31]  # Excel sheet names must be ≤ 31 characters
        site_df = combined_df[combined_df["site"] == site]
        site_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ Exported to {output_path} with one sheet per site.")


✅ Exported to freight_rates_by_site.xlsx with one sheet per site.
